In [449]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torch_geometric.nn as gnn
import pickle
from tqdm import tqdm
import numpy as np
from torch.functional import F
import matplotlib.pyplot as plt
import math


with open("data/train_graphs_nostd_data.pkl", "rb") as file:
    graphs = pickle.load(file)
    file.close()

In [450]:
len(graphs), graphs[0]

(90,
 Data(x=[1140, 33], edge_index=[2, 11709], edge_attr=[11709, 2], y=[1140, 2]))

In [451]:
graphs[0].y[0], graphs[-1].y[0]

(tensor([69.3060, 63.7800]), tensor([66.5690, 62.3200]))

In [452]:
# 划分训练集和验证集，使用sklearn的train_test_split函数
from sklearn.model_selection import train_test_split

train_graphs, val_graphs = train_test_split(graphs, test_size=0.2)
val_graphs, test_graphs = train_test_split(val_graphs, test_size=0.5)
len(train_graphs), len(val_graphs), len(test_graphs)

(72, 9, 9)

In [453]:
class SAGEModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(SAGEModel, self).__init__()

        self.graph_features = nn.ModuleList(
            [
                gnn.SAGEConv(input_size, 8),
                nn.LayerNorm(8),
                nn.ReLU(),
                gnn.SAGEConv(8, 4),
                nn.LayerNorm(4),
                nn.ReLU(),
            ]
        )

        # 考虑更多的trick，如layernorm等等
        self.regression = nn.Sequential(
            nn.Linear(4, output_size),
        )

    def forward(self, data):
        x, edge_index, _ = data.x, data.edge_index, data.edge_attr

        output = x

        for layer in self.graph_features:
            if isinstance(layer, gnn.SAGEConv):
                output = layer(output, edge_index)
            else:
                output = layer(output)

        output = self.regression(output)

        return output

In [454]:
# 看看模型的输入输出
model = SAGEModel(input_size=33, output_size=2)
output = model(graphs[0])
output.shape

torch.Size([1140, 2])

In [455]:
input_size = graphs[0].x.shape[1]
# 这里的output_size就是最终的输出维度，不需要再乘以num_heads
output_size = graphs[0].y.shape[1]
# 个人认为，注意力头的个数应该和输出维度保持一致，因为每个输出可能需要关注不同的邻居
num_heads = graphs[0].y.shape[1]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# batch_size = 256

model_path = "saved/models/sage_nostd_simple.pth"

In [456]:
def calScore(loss):
    return 1 / (1 + math.sqrt(2 * loss))

# 开始训练

In [457]:
def train(
    model, criterion, optimizer, scheduler, train_graphs, val_graphs=None, num_epochs=50
):
    # 训练过程记录
    train_loss_list = []
    val_loss_list = []

    with tqdm(total=num_epochs, desc="Training Progress", unit="epoch") as pbar_epochs:
        for epoch in range(num_epochs):
            # 训练
            model.train()
            train_loss = 0.0

            # 目前每个batch是1个graph
            for i, graph in enumerate(train_graphs):
                graph = graph.to(device)

                output = model(graph)

                optimizer.zero_grad()
                loss = criterion(output, graph.y)
                loss.backward()
                optimizer.step()

                train_loss += loss.item()

            train_loss /= len(train_graphs)
            train_loss_list.append(train_loss)

            # 验证
            if val_graphs:
                model.eval()
                with torch.no_grad():
                    val_loss = 0.0
                    for i, graph in enumerate(val_graphs):
                        graph = graph.to(device)
                        output = model(graph)
                        loss = criterion(output, graph.y)
                        val_loss += loss.item()
                    val_loss /= len(val_graphs)
                    val_loss_list.append(val_loss)

            if val_graphs:
                pbar_epochs.set_postfix(
                    {
                        "train MSE Loss": train_loss,
                        "train score": calScore(train_loss),
                        "val MSE Loss": val_loss,
                        "val score": calScore(val_loss),
                    }
                )
            else:
                pbar_epochs.set_postfix(
                    {"train MSE Loss": train_loss, "train score": calScore(train_loss)}
                )
            pbar_epochs.update(1)
            scheduler.step()

    # 可视化训练过程
    plt.figure()
    plt.plot(train_loss_list, label="train loss")
    if val_graphs:
        plt.plot(val_loss_list, label="val loss")
    plt.xlabel("epoch")
    plt.ylabel("MSE loss")
    plt.ylim(0, 3)
    plt.legend()

    return model

In [458]:
# 定义模型、算法、损失函数
model = SAGEModel(
    input_size=input_size,
    output_size=output_size,
).to(device)
# 考虑是否加入weight_decay
optimizer = optim.Adam(model.parameters(), lr=1e-2, weight_decay=1e-2)
# 学习率衰减
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)
criterion = nn.MSELoss().to(device)

model = train(
    model, criterion, optimizer, scheduler, train_graphs, val_graphs, num_epochs=90
)

Training Progress:  52%|█████▏    | 47/90 [00:15<00:14,  2.96epoch/s, train MSE Loss=1.77, train score=0.347, val MSE Loss=1.73, val score=0.349]       

In [ ]:
# 看看测试集上的效果
def predict(model, test_graphs):
    model.eval()
    with torch.no_grad():
        test_loss = 0.0
        for i, graph in enumerate(test_graphs):
            graph = graph.to(device)
            output = model(graph)
            loss = criterion(output, graph.y)
            test_loss += loss.item()
        test_loss /= len(test_graphs)
    return test_loss, calScore(test_loss)

In [ ]:
predict(model, test_graphs)

(0.8128637340333726, 0.43955267581931606)

In [364]:
# 定义模型、算法、损失函数
model = SAGEModel(
    input_size=input_size,
    output_size=output_size,
).to(device)
# 考虑是否加入weight_decay
optimizer = optim.Adam(model.parameters(), lr=1e-2, weight_decay=1e-2)
# 学习率衰减
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)
criterion = nn.MSELoss().to(device)

model = train(
    model,
    criterion,
    optimizer,
    scheduler,
    train_graphs + val_graphs,
    test_graphs,
    num_epochs=90,
)

Training Progress:  40%|████      | 28/70 [00:11<00:16,  2.50epoch/s, train MSE Loss=1.52, train score=0.364]   


KeyboardInterrupt: 

In [290]:
# 保存模型
torch.save(model.state_dict(), model_path)

# 预测

In [13]:
# 读取模型
model = SAGEModel(
    input_size=input_size,
    output_size=output_size,
).to(device)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [291]:
# 读取node_ids
with open("data/node_ids.pkl", "rb") as file:
    node_ids = pickle.load(file)
    file.close()

In [292]:
# 读取测试集
with open("data/test_graphs_nostd_data.pkl", "rb") as file:
    test_graphs = pickle.load(file)
    file.close()

In [293]:
len(test_graphs), test_graphs[0]

(4, Data(x=[1140, 33], edge_index=[2, 21018], edge_attr=[21018, 2]))

预测

In [294]:
model.eval()
output_list = []
with torch.no_grad():
    for i, graph in enumerate(test_graphs):
        graph = graph.to(device)
        output = model(graph)
        output_list.append(output.detach().cpu().numpy())

In [295]:
len(output_list), output_list[0].shape, output_list[0][0]

(4, (1140, 2), array([68.96431 , 61.401756], dtype=float32))

In [61]:
# 看看是不是缩放完成啦
output_list[0][0]

array([68.632545, 60.75451 ], dtype=float32)

### 把输出转成需要的格式

In [296]:
import csv

# 结果写入csv，分隔符为\t
output_path = "output/sage_nostd_simple_out.csv"

date_id = [20230404, 20230405, 20230406, 20230407]
with open(output_path, "w", newline="") as f:
    writer = csv.writer(f, delimiter="\t")  # 设置分隔符为制表符
    writer.writerow(["geohash_id", "consumption_level", "activity_level", "date_id"])

    # output_list.shape = [4天, 节点数, 2个输出]
    for nidx in range(len(node_ids)):
        for day in range(len(date_id)):
            # 注意不要写反了
            writer.writerow(
                [
                    node_ids[nidx],
                    output_list[day][nidx][1],
                    output_list[day][nidx][0],
                    date_id[day],
                ]
            )
    f.close()